# 1 Data Download

In [1]:
# Download Dataset
import requests
import zipfile
from pathlib import Path
import gdown

# Setup paths and folders names and urls
data_path = Path("data/")
image_path = data_path / "rotated_cropped/"

dataset_url = 'https://drive.google.com/uc?id=1M_UZ1tfAyssGO5_L1K2nMa-YFS4c-xMT'
train_url = 'https://drive.google.com/uc?id=1bI6yuNlP6-NqMup_fWEouIonRfcQq9u5'


# If the image folder doesn't exist, download it
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    data_path.mkdir(parents=True, exist_ok=True)

    print("Downloading train.csv...")
    gdown.download(train_url, str(data_path / "train.csv"), quiet=False)

    print("Downloading Dataset...")
    zip_path = str(data_path / "rotated_cropped.zip")
    gdown.download(dataset_url, zip_path, quiet=False)

    # Unzip data
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        print("Unzipping dataset data...")
        zip_ref.extractall(image_path)

data/rotated_cropped directory exists.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 2 Data Import

In [3]:
import pandas as pd
import numpy as np
import io
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
try:
  from efficientnet_pytorch import EfficientNet
except Exception:
  !pip install efficientnet_pytorch
  from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image

In [4]:
# Path to the image directory and CSV file
train_dir = "./data/rotated_cropped"
csv_file = "./data/train.csv"

# Reading the CSV file using Pandas
data = pd.read_csv(csv_file)
data.head()

,Image,whaleID
0,w_7812.jpg,whale_48813
1,w_4598.jpg,whale_09913
2,w_3828.jpg,whale_45062
3,w_8734.jpg,whale_74162
4,w_3251.jpg,whale_99558


In [5]:
# Build a whale id lookup dict with the respective probability tensors
n = data['whaleID'].nunique()

whale_id_dict = {}

for idx, whale_id in enumerate(sorted(data['whaleID'].unique())):
    tensor = torch.zeros(n, dtype=torch.float32)
    tensor[idx] = 1
    whale_id_dict[whale_id] = tensor

def get_whale_tensor(whale_id):
    return whale_id_dict[whale_id]

def load_image(img_name, folder):
    img_path = os.path.join(folder, img_name)
    if not os.path.exists(img_path):
        return None
    return Image.open(img_path).convert('RGB')

# Sparse Tensor with only one 1 set
get_whale_tensor('whale_06967').shape

#
data_clean = data[data['Image'].isin(os.listdir(train_dir))]
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4206 entries, 0 to 4543
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Image    4206 non-null   object
 1   whaleID  4206 non-null   object
dtypes: object(2)
memory usage: 98.6+ KB


## 3 Dataset and Initialisation

In [6]:
NUM_CLASSES = 447

# Define the Dataset class
class WhaleDataset:
    def __init__(self, dataframe, img_folder, whale_tensor_dict, transform=None):
        self.dataframe = dataframe
        self.img_folder = img_folder
        self.whale_tensor_dict = whale_tensor_dict
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_name = row['Image']
        whale_id = row['whaleID']

        try:
            img_path = os.path.join(self.img_folder, image_name)
            img = Image.open(img_path).convert('RGB')

            if self.transform:
                img = self.transform(img)

            label = self.whale_tensor_dict[whale_id]
            return img, label
        except Exception as e:
            print(e)
            print(whale_id, image_name)

# Define the EfficientNet model
class EfficientNetModel(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(EfficientNetModel, self).__init__()
        # Use 'efficientnet-b0' as a base model
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')

        # Adjust the input size
        self.efficientnet._conv_stem = nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)

        # Adjust the output layer
        in_features = self.efficientnet._fc.in_features
        self.efficientnet._fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.efficientnet(x).softmax(dim=1)

In [7]:
BATCH_SIZE = 24

# Set random seed for reproducibility
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load the dataset and split it into training and validation sets
train_data, valid_data = train_test_split(data_clean, test_size=0.2, random_state=42)

# Create dataset and dataloaders
train_dataset = WhaleDataset(train_data, train_dir, whale_id_dict, transform)
valid_dataset = WhaleDataset(valid_data, train_dir, whale_id_dict, transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the model, loss function, and optimizer
model = EfficientNetModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Loaded pretrained weights for efficientnet-b0


### Train Model

In [ ]:
NUM_EPOCHS = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(model, train_loader, criterion, optimizer, num_epochs=NUM_EPOCHS, device=device):
    model.train()
    model.to(device)

    for epoch in range(num_epochs):
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Example usage
train(model, train_loader, criterion, optimizer, num_epochs=10)

In [ ]:
throw Exception

### Evaluation

In [ ]:
# Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
inputs

In [ ]:
model_save_name = 'classifier.pt'
path = F"/content/{model_save_name}"
torch.save(model.state_dict(), path)

In [ ]:
model = EfficientNetModel(NUM_CLASSES)
model.load_state_dict(torch.load(path))
model.eval()

In [ ]:
model = model.to(device)
inputs = inputs.to(device)
model(inputs)